In [88]:
from scipy.misc import imread
from scipy.misc import imresize
import os
ls=[]
folder='wwe'
files=os.listdir(folder)
files=list(map(lambda x: os.path.join(folder,x),files))
a=(len(files))
for i in range(a):
    im = imread(files[i])
    im=imresize(im,[32,32,3])
    im=im.flatten()
    ls.append(im)

  

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


In [89]:
print(len(ls))

2370


90


In [92]:
import numpy as np

raw_data=np.array(ls,dtype=object)
print(raw_data.shape)

(2370, 3072)


In [93]:
print(raw_data[200])

[163 113 18 ... 242 18 0]


In [95]:
from sklearn.model_selection import train_test_split
train_data,test_data=train_test_split(raw_data,train_size=0.7,test_size=0.3,shuffle=True)
print(train_data.shape)

(1659, 3072)


In [96]:
print(train_data[200])

[234 233 249 ... 203 217 243]


In [97]:
import pickle

with open('data_batch_1', mode='rb') as file:
    batch=pickle.load(file,encoding='bytes')
    data2=batch[b'data']
#num_image=len(features)
print((data2.shape))

(10000, 3072)


In [98]:
beg=10000
end=len(data2) + len(train_data)
#print(end)
images=np.zeros(shape=[end+1,3072])
images[0:beg, :]=data2
images[beg:end,:]=train_data
images[end]=images[11330]
print(images.shape)

(11660, 3072)


In [0]:
images=images.reshape(-1,32,32,3)
images=images/255.0
#print(images[2])

In [101]:
print(test_data[201])

[245 254 253 ... 245 255 249]


In [103]:
with open('data_batch_2', mode='rb') as file:
    batch=pickle.load(file,encoding='bytes')
    tes=batch[b'data']
#num_image=len(features)
#print((tes.shape))
tes=tes[0:5000,:]
print(tes.shape)
end=len(tes) + len(test_data)
test_images=np.zeros(shape=[end,3072])
beg=5000
test_images[0:beg,:]=tes
test_images[beg:end,:]=test_data
print(test_images[5201])
print(test_images.shape)

(5000, 3072)
[245. 254. 253. ... 245. 255. 249.]
(5711, 3072)


In [105]:
test_images=test_images.reshape(-1,32,32,3)
print(test_images.shape)
test_images=test_images/255.0

(5711, 32, 32, 3)


In [106]:
label_train=np.zeros(shape=[11660,1])
label_train[10000:11660,:]=1
print(label_train.shape)

(11660, 1)


In [109]:
label_test=np.zeros(shape=[5711,1])
label_test[5000:5711,:]=1
print(label_test[5001])

[1.]


In [110]:
a=(images.shape[0])
i=np.random.permutation(a)
x_train,y_train=images[i],label_train[i]
print(y_train.shape)

(11660, 1)


In [165]:
j=np.random.permutation(test_images.shape[0])
data_test,label_test=test_images[j],label_test[j]
print(data_test.shape)

(5711, 32, 32, 3)


In [0]:
def one_hot(raw_labels):
    import numpy as np
    raw_labels=np.array(raw_labels,np.int32)
    n_values = np.max(raw_labels) + 1
    onehotDash=np.eye(n_values)[raw_labels]
    onehot=np.reshape(onehotDash,(len(raw_labels),2))##we reshape from a 3D matrix as built by default using eye function to a 2D array
    print(onehot.shape)
    return onehot

In [136]:
onehot_train=one_hot(y_train)

(11660, 2)


In [164]:
onehot_test=one_hot(label_test)

(5711, 2)


In [140]:
#val_x,x_test,raw_val_y,raw_y_test=train_test_split(data_test,label_test,train_size=0.6,test_size=0.4,shuffle=True)
#print(raw_y_test.shape)

(2285, 1)


289


(3426, 2)
(2285, 2)
(2285, 32, 32, 3)


(11660, 2)


In [0]:
def weight_init(shape): 
    weight=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(weight)

In [0]:
def bias_init(shape):
    bias=tf.constant(0.2,shape=shape)
    return tf.Variable(bias)

In [0]:
def conv_d(x,w):
    return tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')

In [0]:
def max_pool(x):
      return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [0]:
import tensorflow as tf
x=tf.placeholder(tf.float32,shape=[None,32,32,3])
y_=tf.placeholder(tf.float32,shape=[None,2])

In [210]:
w_conv1=weight_init([5,5,3,16])
b_conv1=bias_init([16])
#x_final=convert_images(x)
f_conv=tf.nn.relu(conv_d(x,w_conv1) + b_conv1)
#batch_mean2, batch_var2 = tf.nn.moments(f_conv,[0])


f_pool=max_pool(f_conv)
w_conv2=weight_init([5,5,16,32])
b_conv2=bias_init([32])
s_conv=tf.nn.relu(conv_d(f_pool,w_conv2) + b_conv2)
s_pool=max_pool(s_conv)
w_conv3=weight_init([5,5,32,64])
b_conv3=bias_init([64])
t_conv=tf.nn.relu(conv_d(s_pool,w_conv3) + b_conv3)
t_pool=max_pool(t_conv)
weight_flat=weight_init([4*4*64,10])
bias_flat=bias_init([10])
t_pool_flat=tf.reshape(t_pool,[-1,4*4*64])

keep_prob=tf.placeholder(tf.float32)
scale1 = tf.Variable(tf.ones([10]))
shift1 = tf.Variable(tf.zeros([10]))

f1=(tf.matmul(t_pool_flat,weight_flat)+bias_flat)
mean1, var1 = tf.nn.moments(f1, [0])
h_fc1_normed = tf.nn.batch_normalization(f1,mean1,var1,shift1,scale1,1e-4)
h_fc1=tf.nn.relu(h_fc1_normed)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob)
weight_flat2=weight_init([10,10])
bias_flat2=bias_init([10])
scale2 = tf.Variable(tf.ones([10]))
shift2 = tf.Variable(tf.zeros([10]))

f2=(tf.matmul(h_fc1_drop,weight_flat2)+bias_flat2)
mean2, var2 = tf.nn.moments(f2, [0])
h_fc2_normed = tf.nn.batch_normalization(f2,mean2,var2,shift2,scale2,1e-4)
h_fc2=tf.nn.relu(h_fc2_normed)
h_fc2_drop=tf.nn.dropout(h_fc2,keep_prob)
weight_flat3=weight_init([10,2])
bias_flat3=bias_init([2])
y_conv=(tf.matmul(h_fc2_drop,weight_flat3)+bias_flat3)

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)


correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    batch=20
    for j in range(8):

        for i in range(583):


            batch_labels=onehot_train[i*batch:(i+1)*batch,:]

            batch_images=x_train[i*batch:(i+1)*batch,:]
            train_step.run(feed_dict={x:batch_images , y_:batch_labels , keep_prob: 0.15}) 

        train_accuracy = accuracy.eval(feed_dict={x:x_train, y_:onehot_train, keep_prob: 1.0})
        print("epoch %d, training accuracy %g"%(j, train_accuracy))
        print(sess.run(cross_entropy,feed_dict={x:x_train,y_:onehot_train, keep_prob: 1.0}))                                          

    print("test accuracy %g"%accuracy.eval(feed_dict={
      x:data_test , y_: onehot_test, keep_prob: 1.0})

epoch 0, training accuracy 0.87753
0.16509742
epoch 1, training accuracy 0.933448
0.13113108
epoch 2, training accuracy 0.915437
0.14457989
epoch 3, training accuracy 0.952916
0.10798468
epoch 4, training accuracy 0.972642
0.094128735
epoch 5, training accuracy 0.978816
0.083236754
epoch 6, training accuracy 0.957633
0.100430146
epoch 7, training accuracy 0.960978
0.10937682
test accuracy 0.857468
